In [ ]:
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D

allow_pickle=True
groups_folder_path = '/content/drive/My Drive/Colab Notebooks/workspace/'
categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
num_classes = len(categories)

image_w = 64
image_h = 64
  
X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'

    for top, dirs, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = cv2.imread(image_dir+filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            X.append(img/256)
            Y.append(label)
 
X = np.array(X)
Y = np.array(Y)
 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)
 
np.save("/content/drive/My Drive/Colab Notebooks/workspace/img_data.npy", xy)

Using TensorFlow backend.


/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (402).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (87).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (625).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (103).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (470).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (263).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (74).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (430).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (364).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (304).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (412).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (217).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (226).jpg
/content/drive/My Drive/Colab Notebooks/workspace/daisy/daisy (264

In [ ]:
X_train, X_test, Y_train, Y_test = np.load('/content/drive/My Drive/Colab Notebooks/workspace/img_data.npy', allow_pickle=True)
 
model = Sequential()
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu',
                        input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
  
model.add(Convolution2D(64, 3, 3,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
  
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation = 'softmax'))
  
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, nb_epoch=100)
 
model.save('flowers')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(64, 64, 3..., padding="same")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
2748/2748 [==============================] - 16s 6ms/step - loss: 0.4704 - accuracy: 0.8017
Epoch 2/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.3884 - accuracy: 0.8229
Epoch 3/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.3471 - accuracy: 0.8477
Epoch 4/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.3295 - accuracy: 0.8547
Epoch 5/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.3126 - accuracy: 0.8635
Epoch 6/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.2903 - accuracy: 0.8745
Epoch 7/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.2703 - accuracy: 0.8861
Epoch 8/100
2748/2748 [==============================] - 14s 5ms/step - loss: 0.2613 - accuracy: 0.8878
Epoch 9/100
2748/2748 [==============================] - 15s 5ms/step - loss: 0.2539 - accuracy: 0.8924
Epoch 10/100
2748/2748 [==============================] - 14s 5m

In [ ]:
X_train = np.append(X_train, X_test, axis=0)
Y_train = np.append(Y_train, Y_test, axis=0)

# Save Model with CheckPoint & StopPoint
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import datetime
 
Datetime = datetime.datetime.now().strftime('%m%d_%H%M')
modelpath="flowers"
 
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)
 
# Learning and save models
model.fit(X_train, Y_train, validation_split=0.1, epochs=3500, batch_size=10, verbose=0, callbacks=[early_stopping_callback,checkpointer])


Epoch 00001: loss improved from inf to 0.22494, saving model to flowers

Epoch 00002: loss improved from 0.22494 to 0.14811, saving model to flowers

Epoch 00003: loss improved from 0.14811 to 0.11384, saving model to flowers

Epoch 00004: loss improved from 0.11384 to 0.10219, saving model to flowers

Epoch 00005: loss improved from 0.10219 to 0.09038, saving model to flowers

Epoch 00006: loss did not improve from 0.09038

Epoch 00007: loss improved from 0.09038 to 0.07848, saving model to flowers

Epoch 00008: loss did not improve from 0.07848

Epoch 00009: loss did not improve from 0.07848

Epoch 00010: loss did not improve from 0.07848

Epoch 00011: loss did not improve from 0.07848

Epoch 00012: loss improved from 0.07848 to 0.07572, saving model to flowers

Epoch 00013: loss improved from 0.07572 to 0.06470, saving model to flowers

Epoch 00014: loss did not improve from 0.06470

Epoch 00015: loss improved from 0.06470 to 0.06428, saving model to flowers

Epoch 00016: loss impr

In [ ]:
import os, re, glob
import cv2
import numpy as np
import shutil
from numpy import argmax
from keras.models import load_model
 
categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
 
def Dataization(img_path):
    image_w = 64
    image_h = 64
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
    return (img/256)
 
src = []
name = []
test = []
image_dir = "/content/drive/My Drive/Colab Notebooks/workspace/test/"
for file in os.listdir(image_dir):
    if (file.find('.jpg') is not -1):      
        src.append(image_dir + file)
        name.append(file)
        test.append(Dataization(image_dir + file))
 
 
test = np.array(test)
model = load_model('flowers')
predict_c = model.predict_classes(test) #flower name
predict_acc = model.predict(test) #flower accurancy
for i in range(len(test)):
    print(name[i] + " : , Predict : "+ str(categories[predict_c[i]]), ", accurancy : " + str(round(max((predict_acc[i]) * 100), 3)) + "%")

tulip_test.jpg : , Predict : roses , accurancy : 99.991%
daisy_test.jpg : , Predict : daisy , accurancy : 95.573%
dandelion_test.jpg : , Predict : dandelion , accurancy : 100.0%
rose_test.jpg : , Predict : roses , accurancy : 99.993%
sunflower_test.jpg : , Predict : sunflowers , accurancy : 100.0%
carnation_test.jpg : , Predict : roses , accurancy : 100.0%
sakura_test.jpg : , Predict : daisy , accurancy : 77.496%
forever_flower_test.jpg : , Predict : daisy , accurancy : 44.092%
nationalflower_test.jpg : , Predict : roses , accurancy : 99.538%
hydrangea_test.jpg : , Predict : roses , accurancy : 78.669%
sunflower_painting_test.jpg : , Predict : sunflowers , accurancy : 100.0%
